In [8]:
import os
import numpy as np
import scipy.io as sio
import mne
from scipy.signal import resample

In [ ]:
SAMPLE_RATE = 128  # fs
SAMPLE_LEN = 128   # T

## Official script

In [9]:
filenames = []
for filename in os.listdir("APAVA/"):
  filenames.append(filename)

In [10]:
filenames.sort()
# filenames

In [11]:
feature_path = 'Processed/APAVA/Feature'
if not os.path.exists(feature_path):
    os.makedirs(feature_path)

#### Save feature

In [12]:
def resample_time_series(data, original_fs, target_fs):
    T, C = data.shape
    new_length = int(T * target_fs / original_fs)
    
    resampled_data = np.zeros((new_length, C))
    for i in range(C):
        resampled_data[:, i] = resample(data[:, i], new_length)
        
    return resampled_data

In [13]:
subseq_length = SAMPLE_LEN
stride = SAMPLE_LEN / 2  # Half of the subsequence length for half-overlapping
for i in range(len(filenames)):
    # print('Dataset/'+filename)
    path = "APAVA/" + filenames[i]
    mat = sio.loadmat(path)
    mat_np = mat['data']

    # Get epoch number for each subject
    epoch_num = len(mat_np[0,0][2][0])
    print("Epoch number: ",epoch_num)
    # Each epoch has shape (1280, 16)
    raw_shape = np.zeros((epoch_num, 1280, 16)).shape
    features = []
    # Store in temp
    for j in range(epoch_num):
        temp = np.transpose(mat_np[0,0][2][0][j])
        data = resample_time_series(temp, 256, SAMPLE_RATE)  # Downsample to 128 Hz
        # Calculate the number of subsequences that can be extracted
        num_subsequences = (data.shape[0] - subseq_length) // stride + 1
        # Extract the subsequences
        subsequences = [data[i * stride : i * stride + subseq_length, :] for i in range(num_subsequences)]
        feature = np.array(subsequences)
        features.append(feature)
    features = np.array(features).reshape((-1, subseq_length, 16))   # Reshape to (N, 128, 16)

    print(f"Filename: {filenames[i]}")
    print(f"Patient ID: {i+1}")
    print("Raw data:", raw_shape)
    print("Downsampling, segmented and interpolated data", features.shape)
    np.save(feature_path + "/feature_{:02d}.npy".format(i+1),features)
    print("Save feature_{:02d}.npy".format(i+1))
    print("---------------------------------------------\n")

Epoch number:  35
Filename: preproctrials01.mat
Patient ID: 1
Raw data: (35, 1280, 16)
Downsampling, segmented and interpolated data (315, 128, 16)
Save feature_01.npy
---------------------------------------------

Epoch number:  25
Filename: preproctrials02.mat
Patient ID: 2
Raw data: (25, 1280, 16)
Downsampling, segmented and interpolated data (225, 128, 16)
Save feature_02.npy
---------------------------------------------

Epoch number:  10
Filename: preproctrials03.mat
Patient ID: 3
Raw data: (10, 1280, 16)
Downsampling, segmented and interpolated data (90, 128, 16)
Save feature_03.npy
---------------------------------------------

Epoch number:  33
Filename: preproctrials04.mat
Patient ID: 4
Raw data: (33, 1280, 16)
Downsampling, segmented and interpolated data (297, 128, 16)
Save feature_04.npy
---------------------------------------------

Epoch number:  1
Filename: preproctrials05.mat
Patient ID: 5
Raw data: (1, 1280, 16)
Downsampling, segmented and interpolated data (9, 128, 1

#### Save label

In [14]:
AD_positive = [1,3,6,8,9,11,12,13,15,17,19,21]

In [15]:
labels = np.zeros((23, 2))
len(labels)

23

In [16]:
label_path = 'Processed/APAVA/Label'
if not os.path.exists(label_path):
    os.makedirs(label_path)

In [17]:
for i in range(len(labels)):
  # The first one is AD label (0 for healthy; 1 for AD patient)
  # The second one is the subject label (the order of subject, ranging from 1 to 23.
  labels[i][1] = i + 1
  if i+1 in AD_positive:
    labels[i][0] = 1
  else:
    labels[i][0] = 0

In [18]:
np.save(label_path + "/label.npy",labels)
print("Save label")

Save label


## Test

In [19]:
# Test the saved npy file
# example

path = 'Processed/APAVA/Feature/'

for file in os.listdir(path):
    sub_path = os.path.join(path, file)
    print(np.load(sub_path).shape)

(315, 128, 16)
(225, 128, 16)
(90, 128, 16)
(297, 128, 16)
(9, 128, 16)
(198, 128, 16)
(27, 128, 16)
(288, 128, 16)
(162, 128, 16)
(342, 128, 16)
(423, 128, 16)
(333, 128, 16)
(261, 128, 16)
(351, 128, 16)
(414, 128, 16)
(252, 128, 16)
(531, 128, 16)
(360, 128, 16)
(414, 128, 16)
(333, 128, 16)
(171, 128, 16)
(153, 128, 16)
(18, 128, 16)


In [20]:
np.load("Processed/APAVA/Label/label.npy")

array([[ 1.,  1.],
       [ 0.,  2.],
       [ 1.,  3.],
       [ 0.,  4.],
       [ 0.,  5.],
       [ 1.,  6.],
       [ 0.,  7.],
       [ 1.,  8.],
       [ 1.,  9.],
       [ 0., 10.],
       [ 1., 11.],
       [ 1., 12.],
       [ 1., 13.],
       [ 0., 14.],
       [ 1., 15.],
       [ 0., 16.],
       [ 1., 17.],
       [ 0., 18.],
       [ 1., 19.],
       [ 0., 20.],
       [ 1., 21.],
       [ 0., 22.],
       [ 0., 23.]])